In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import nltk  
import numpy as np  
import random  
import string
import re
from itertools import repeat

In [ ]:
df = pd.DataFrame()
df  = pd.read_csv('labels_imperium_all.csv',encoding='utf-8',header=None)
labels = df[0]
df.head()

In [ ]:
df = pd.DataFrame()
df  = pd.read_csv('imperium_txts_all.csv',encoding='utf-8',header=None)
df.head()

In [ ]:
documents = df[0]
print(documents[0])
# lower case
documents = documents.str.lower() 
# remove punctuation
documents = documents.str.replace('[^\w\s]','') 
documents = documents.values.tolist()
for i in range(len(documents )):
    # tokenize
    documents [i] = nltk.word_tokenize(documents [i])
    # remove blanck spaces
    documents [i] = [re.sub(r'\W',' ',word) for word in documents [i] ]
print("Ther are :",len(documents)," 0:", documents[0])

Cleaning the data: Remove puntuation

In [ ]:
wordfreq = {}
for sentence in documents:
    for token in sentence:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1
print("Ther are :",len(wordfreq)," unique words")

The wordfreq save each unique word and the number of time that his words appears

In [ ]:
# Use vectorize function of numpy 
length_checker = np.vectorize(len) 
  
# Find the length of each element 
documents_length = length_checker(documents) 
  
# Print the length of each element 
print(documents_length) 

In [ ]:
# List of keys to be deleted from dictionary
selectedKeys = list() 
 
# Iterate over the dict and put to be deleted keys in the list
for (key, value) in wordfreq.items():
    if value < 5:
        selectedKeys.append(key)
    if len(key) < 3:
        selectedKeys.append(key)
    if len(key) > 30:
        selectedKeys.append(key)
 
# Iterate over the list and delete corresponding key from dictionary
for key in selectedKeys:
    if key in wordfreq:
        del wordfreq[key]
print("Ther are :",len(wordfreq)," unique words")

In [ ]:
# Saving each unique word index
word_idx = {}
idx = 0
for token in wordfreq:
    word_idx[token] =  idx
    idx += 1
print(word_idx['florida'])

In [ ]:
sentence_vectors = []
for sentence in documents:
    sent_vec = []
    for token in wordfreq:
        if token in sentence:
            df = sentence.count(token)
            sent_vec.append(df)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)

In [ ]:
sentence_vectors = np.asarray(sentence_vectors)
print("Matrix of dimension: (",sentence_vectors.shape[0],",",sentence_vectors.shape[1],")")

In [ ]:
# Identifying the document with more terms
maxlen_doc = []
for doc in documents:
    maxlen_doc.append(len(doc))
print("Maximum document longitud: ",max(maxlen_doc))  
print(max(documents_length))
maxlen_doc = (max(documents_length))

Creating an document / index matrix

In [ ]:
# We need to separete this in sTrain and Stest, we need to be careful to mantein the index in the training fase
num_docs = sentence_vectors.shape[0]
#maximum = []
idx_matriz = np.zeros((num_docs, maxlen_doc)) # idx_matriz represents "stest" in Matlab
row = 0
for doc in documents:
    # If tern not in dicc, add '0'
    doc_inx = [ word_idx[x] if x in word_idx else 0 for x in doc] 
    doc_inx.extend(repeat(0, maxlen_doc - len(doc_inx)))  # Here we are padding, but it is NOT necessary
    idx_matriz[row] = (doc_inx)
    row += 1
    # Like this ingnore those words tht are not in dic : doc_inx = [wordfreq[x] for x in doc if x in wordfreq]
    #maximum.append(len(doc_inx))
print("Matrix of dimension: (",idx_matriz.shape[0],",",idx_matriz.shape[1],")")

In [ ]:
#np.savetxt("document_matrix.csv", sentence_vectors, delimiter=",")

In [ ]:
#df  = pd.read_csv('document_matrix.csv',encoding='utf-8',header=None)
#df.head()

# SEPARAR SIN PERDER

In [ ]:
X_train = sentence_vectors[:6182]
X_test  = sentence_vectors[6182:]
Y_train = labels[:6182]
Y_test  = labels[6182:]
S_train = idx_matriz[:6182]
S_test  = idx_matriz[6182:]    
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(S_train.shape)
print(S_test.shape)

In [ ]:
def csa(X_train,Y,X_test):
    num_terms = X_train.shape[1]
    num_class = 2
    dl_train = documents_length [:6182]
    dl_test  = documents_length [6182:] 
    TR = np.zeros((num_terms, num_class))
    num_docs = X_train.shape[0]
    
    # Non - Normalization 0'
    for j in range (0,num_class): # j = [0;1]
        docs = np.where(Y == j)[0] 
        for i in range(0,num_terms):
            tf_v = X_train[docs,i] # tf the term (v)
            TR[i][j] += sum(np.log2(1 + tf_v / dl_train [docs]))

    #--print("PRB: (",TR.shape[0],",",TR.shape[1],")")
    TR = np.nan_to_num(TR)
    # Normalization 1' Sum all the weights vectors 
    n1 = np.sum(TR, axis = 0) #Sum by colum
    Tik = TR / n1
    Tik = np.nan_to_num(Tik)
    # Normalization 2' Each componet of tik [ti1,ti2]/ [ti1+ti2;ti1+ti2]
    for i in range(0,num_terms):
        Tik[i] = Tik[i] / sum(Tik[i]) # Tik[i] = Tik[i] / sum(TR[i])

    print("Tik: (",Tik.shape[0],",",Tik.shape[1],")")
    Tik = np.nan_to_num(Tik)
    # Document Representation
    DR = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(X_train[i] != 0)[0] # extrac idxs =! 0
        a = X_train[i][termos] / dl_train[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DR[i] = sum(np.multiply(a,Tik[termos]))
        
    # Document Test Representation
    num_docs = X_test.shape[0]
    DRT = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(X_test[i] != 0)[0] # extrac idxs =! 0
        a = X_test[i][termos] / dl_test[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DRT[i] = sum(np.multiply(a,Tik[termos]))

    return DR,DRT

In [ ]:
DR,DRT = csa(X_train,Y_train,X_test)
DR = np.nan_to_num(DR)
DRT = np.nan_to_num(DRT)
print(DR.shape)
print(DRT.shape)

In [ ]:
for i in DRT:
    print(i)

In [ ]:
len(documents_length)

PBR: u did this for all X, u shoul do this 4 xtrain and x test

In [ ]:
"""num_terms = sentence_vectors.shape[1]
num_class = 2
TR = np.zeros((num_terms, num_class))
num_docs = sentence_vectors.shape[0]
"""

In [ ]:
"""
for j in range (0,num_class): # j = [0;1]
        docs = np.where(labels == j)[0] 
        for i in range(0,num_terms):
            tf_v = sentence_vectors[docs,i]
            TR[i][j] += sum(np.log2(1 + tf_v / documents_length [docs]))

print("PRB: (",TR.shape[0],",",TR.shape[1],")")

# Normalization 1' Sum all the weights vectors 
n1 = np.sum(TR, axis = 0) #Sum by colum
Tik = TR / n1

# Normalization 2' Each componet of tik [ti1,ti2]/ [ti1+ti2;ti1+ti2]
for i in range(0,num_terms):
    Tik[i] = Tik[i] / sum(Tik[i]) # Tik[i] = Tik[i] / sum(TR[i])

print("Tik: (",Tik.shape[0],",",Tik.shape[1],")")
"""

In [ ]:
# Document Representation
"""DR = np.zeros((num_docs, num_class))
for i in range(0,num_docs):
    termos = np.where(sentence_vectors[i] != 0)[0] # extrac idxs =! 0
    a = sentence_vectors[i][termos] / documents_length[i]# acces idxs content / len
    a = np.expand_dims(a, axis=1)
    DR[i] = sum(np.multiply(a,Tik[termos]))
DR.shape
# Some of then are empy sentence_vectors[i]  examp 8825 "Nadie se salva de la regla 34 xd"
# none of the owrds bellow to dicc
"""

In [ ]:
#print("DR: (",DR.shape[0],",",DR.shape[1],")")
#for i in DR:
#    print(i)

In [ ]:
"""
X_train = DR[:6182]
X_test  = DR[6182:]
Y_train = labels[:6182]
Y_test  = labels[6182:]
S_train = idx_matriz[:6182]
S_test  = idx_matriz[6182:]
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(S_train.shape)
print(S_test.shape)
"""

In [ ]:
#X_train, X_test, Y_train, Y_test = train_test_split( DR, labels, test_size=0.3, random_state=42)
#print(X_train.shape)
#print(Y_train.shape)
#print(X_test.shape)
#print(Y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import keras
#create model
num_class = 2
model = Sequential()
#get number of columns in training data
#add model layers
model.add(Dense(10, activation='relu', input_shape=(num_class,)))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

In [ ]:
opt = keras.optimizers.SGD(decay=1e-14)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.fit(DR, Y_train, epochs=100, batch_size=10, validation_data=(DRT, Y_test))

Reading parcial data

In [ ]:
retains = [0.0100000000000000,0.0600000000000000,0.110000000000000,0.160000000000000,0.210000000000000,0.260000000000000,0.310000000000000,0.360000000000000,0.410000000000000,0.460000000000000,0.510000000000000,0.560000000000000,0.610000000000000,0.660000000000000,0.710000000000000,0.760000000000000,0.810000000000000,0.860000000000000,0.910000000000000,1]
#retains = [0.0100000000000000]
for j in range (0,len(retains)):
    print("---------------------------------------- ",retains[j]," ---------------------------------------------")
    rXtest = np.zeros((X_test.shape[0], X_test.shape[1]))
    for i in range(0,X_test.shape[0]): # iterating each test document
        noz = np.where(S_test[i] != 0)[0] # from our idx (ONLY INDXS) document matriz, extract idx != 0 (this step could be avoid if we do not agregate those 0)
        ntermssf = round(len(noz)*retains[j]) # We obtain the number of terns with x% of longitud
        myox = 1 # minnum Termos = 1
        wdix = 0 # 
        freqtsof = np.zeros(X_test.shape[1])  
        # This vector activate the terms present in the Document until that longitud
        while myox <= ntermssf:
            if S_test[i,wdix] != 0:
                freqtsof[int(S_test[i,wdix])] += 1 #If the same word appears more than once , just add one to the position
                myox += 1
            wdix += 1
        rXtest[i] = freqtsof
        
    DRS,DR_TS = csa(X_train,Y_train,rXtest)
    DRS = np.nan_to_num(DRS)
    DR_TS = np.nan_to_num(DR_TS)
    print(DRS.shape)
    print(DR_TS.shape)
    his = model.fit(DRS, Y_train, epochs=100, batch_size=10, validation_data=(DR_TS, Y_test))
    #Xtestv = rXtest

In [ ]:
hacer funcion las representaciones
verificar el pre procesamietno
PBR: u did this for all X, u shoul do this 4 xtrain and x test